In [1]:
%matplotlib inline
import xarray as xr
import scipy as sp
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import os 
import gcsfs

import dask.distributed as dd
import dask
import rhg_compute_tools.kubernetes as rhgk

In [2]:
fs = gcsfs.GCSFileSystem(token='/opt/gcsfuse_tokens/impactlab-data.json')

In [ ]:
client, cluster = rhgk.get_standard_cluster()
cluster

In [ ]:
# ! pip install -e /home/jovyan/downscaling/xclim/

In [3]:
# ! pip install xclim 
from xclim import sdba

In [4]:
from xclim.sdba.adjustment import AnalogQuantilePreservingDownscaling, QuantileDeltaMapping
from xclim.sdba.utils import equally_spaced_nodes
from xclim.core.calendar import convert_calendar
from xclim import set_options

In [5]:
def convert_to_noleap_calendar(ds, target='noleap'):
    ds_noleap = convert_calendar(ds, target=target)
    return ds_noleap

In [6]:
def compute_day_ind(coarse, dim='time', variable='tas', num_quantiles=620): 
    """
    function that implements analog downscaling method by computing day indices that can be used to index into the coarse and fine CDFs 
    coarse: time series of coarse data for a day group for a single gridcell 
    """
    
    x = np.arange(0, num_quantiles)
    coarse_quantiles = x / float(len(x))
    
    # coarse_quantile_values = coarse.quantile(q=coarse_quantiles, dim='time')
    
    day_inds = coarse.argsort(axis=0)
    
    return day_inds

set up test data. Using the full bias correction time period for part of Washington state from ERA-5 1/4 degree data. 

In [ ]:
direc = '/gcs/impactlab-data/climate/source_data/ERA-5/tasmax/daily/netcdf/v1.1'
years = np.arange(1994, 2015)
filenames = [os.path.join(direc, 'tmax_daily_%s-%s.nc' %(year, year)) for year in years]
ref_fine = xr.open_mfdataset(filenames)
ref_fine = convert_calendar(ref_fine, target='noleap')

In [ ]:
# temp = xr.open_mfdataset(filenames)

In [ ]:
'''# small Washington state mask 
min_lon = -124
min_lat = 47
max_lon = -122
max_lat = 49

mask_lon = (temp.longitude >= min_lon) & (temp.longitude <= max_lon)
mask_lat = (temp.latitude >= min_lat) & (temp.latitude <= max_lat)'''

In [ ]:
# temp.coords['longitude'] = (temp.coords['longitude'] + 180) % 360 - 180

In [7]:
'''tas_test = temp.where(mask_lon & mask_lat, drop=True)
tas_eager = tas_test.load()
tas_eager.to_netcdf('/home/jovyan/analog_method_prototype_wastate_era5_data.nc')'''
tas_eager = xr.open_dataset('/home/jovyan/analog_method_prototype_wastate_era5_data.nc')
tas_eager = convert_calendar(tas_eager, target='noleap')

test my `xclim` implementation!!!!!

In [8]:
# using the same gridcell as coarse and fine, just want to see if it runs 
'''ref = tas_eager['tas'][:, 0, 0]
hist = tas_eager['tas'][:, 0, 0] + 3
sim = tas_eager['tas'][:, 0, 0] + 5'''
ref = tas_eager['tas']
hist = tas_eager['tas'] + 3
sim = tas_eager['tas'] + 5

In [9]:
ref

<xarray.DataArray 'tas' (time: 7300, latitude: 9, longitude: 9)>
array([[[268.98083, 273.63644, 275.51755, ..., 272.5698 , 269.59067,
         266.28317],
        [266.85895, 271.55002, 274.66153, ..., 274.6161 , 270.8034 ,
         265.57114],
        [274.66162, 274.96127, 275.51334, ..., 274.46127, 270.97836,
         266.90262],
        ...,
        [276.55045, 273.03738, 272.3305 , ..., 276.2988 , 275.2481 ,
         273.8011 ],
        [277.48276, 276.48953, 275.7801 , ..., 276.4514 , 275.13492,
         273.4787 ],
        [277.26044, 276.4395 , 275.53232, ..., 274.81155, 273.49063,
         271.37643]],

       [[270.01285, 274.20154, 275.6221 , ..., 272.372  , 269.50977,
         266.11362],
        [267.55618, 271.87317, 274.6495 , ..., 274.51498, 270.6453 ,
         265.1697 ],
        [274.51508, 274.86426, 275.3815 , ..., 274.52335, 270.83804,
         266.30008],
...
        [274.2319 , 269.30942, 269.18973, ..., 276.31326, 275.09464,
         271.45358],
        [275.75818, 275.01566, 274.8961 , ..., 276.1039 , 272.65088,
         268.4777 ],
        [276.3003 , 275.52567, 275.01456, ..., 272.75818, 269.28732,
         265.17648]],

       [[270.86246, 272.64957, 273.84088, ..., 272.03262, 267.6401 ,
         263.30533],
        [269.78098, 271.24533, 272.64566, ..., 273.76135, 269.27042,
         262.34512],
        [273.28897, 272.9915 , 273.3652 , ..., 273.73306, 269.2184 ,
         262.75864],
        ...,
        [272.22208, 267.48837, 267.42505, ..., 273.89114, 272.78903,
         268.3098 ],
        [273.10336, 272.3209 , 272.2585 , ..., 274.004  , 270.1412 ,
         264.73102],
        [273.2888 , 272.53244, 271.99747, ..., 270.62155, 266.7418 ,
         262.38528]]], dtype=float32)
Coordinates:
  * longitude  (longitude) float32 -124.0 -123.8 -123.5 ... -122.5 -122.2 -122.0
  * latitude   (latitude) float32 49.0 48.75 48.5 48.25 ... 47.5 47.25 47.0
  * time       (time) object 1995-01-01 00:00:00 ... 2014-12-31 00:00:00

In [ ]:
with set_options(sdba_extra_output=True):
    quantiles = equally_spaced_nodes(620, eps=None)
    QDM = QuantileDeltaMapping(kind='+', nquantiles=quantiles, 
                               group=sdba.Grouper("time.dayofyear", window=31))
    QDM.train(ref, hist)
    fake_biascorrected = QDM.adjust(sim)

In [ ]:
fake_biascorrected

In [ ]:
fake_biascorrected = fake_biascorrected['scen'].assign_coords(sim_q=fake_biascorrected.sim_q).to_dataset()

In [ ]:
if 'sim_q' in fake_biascorrected.coords: 
    print("you got your quantiles in here buddy")

In [10]:
AIQPD = AnalogQuantilePreservingDownscaling(kind='+', nquantiles=620, 
                                            group=sdba.Grouper("time.dayofyear", window=31))

In [14]:
ref

<xarray.DataArray 'tas' (time: 7300, latitude: 9, longitude: 9)>
array([[[268.98083, 273.63644, 275.51755, ..., 272.5698 , 269.59067,
         266.28317],
        [266.85895, 271.55002, 274.66153, ..., 274.6161 , 270.8034 ,
         265.57114],
        [274.66162, 274.96127, 275.51334, ..., 274.46127, 270.97836,
         266.90262],
        ...,
        [276.55045, 273.03738, 272.3305 , ..., 276.2988 , 275.2481 ,
         273.8011 ],
        [277.48276, 276.48953, 275.7801 , ..., 276.4514 , 275.13492,
         273.4787 ],
        [277.26044, 276.4395 , 275.53232, ..., 274.81155, 273.49063,
         271.37643]],

       [[270.01285, 274.20154, 275.6221 , ..., 272.372  , 269.50977,
         266.11362],
        [267.55618, 271.87317, 274.6495 , ..., 274.51498, 270.6453 ,
         265.1697 ],
        [274.51508, 274.86426, 275.3815 , ..., 274.52335, 270.83804,
         266.30008],
...
        [274.2319 , 269.30942, 269.18973, ..., 276.31326, 275.09464,
         271.45358],
        [275.75818, 275.01566, 274.8961 , ..., 276.1039 , 272.65088,
         268.4777 ],
        [276.3003 , 275.52567, 275.01456, ..., 272.75818, 269.28732,
         265.17648]],

       [[270.86246, 272.64957, 273.84088, ..., 272.03262, 267.6401 ,
         263.30533],
        [269.78098, 271.24533, 272.64566, ..., 273.76135, 269.27042,
         262.34512],
        [273.28897, 272.9915 , 273.3652 , ..., 273.73306, 269.2184 ,
         262.75864],
        ...,
        [272.22208, 267.48837, 267.42505, ..., 273.89114, 272.78903,
         268.3098 ],
        [273.10336, 272.3209 , 272.2585 , ..., 274.004  , 270.1412 ,
         264.73102],
        [273.2888 , 272.53244, 271.99747, ..., 270.62155, 266.7418 ,
         262.38528]]], dtype=float32)
Coordinates:
  * longitude  (longitude) float32 -124.0 -123.8 -123.5 ... -122.5 -122.2 -122.0
  * latitude   (latitude) float32 49.0 48.75 48.5 48.25 ... 47.5 47.25 47.0
  * time       (time) object 1995-01-01 00:00:00 ... 2014-12-31 00:00:00

In [11]:
AIQPD.train(ref, hist)
AIQPD

AnalogQuantilePreservingDownscaling(nquantiles=620, kind='+', group=Grouper(add_dims=['window'], name='time.dayofyear', window=31, interp=False))

In [ ]:
AIQPD.ds

In [ ]:
QDM.ds.af

In [ ]:
downscaled = AIQPD.adjust(fake_biascorrected)

In [ ]:
# downscaled

In [ ]:
plt.plot(downscaled)

# plt.plot(fake_biascorrected['scen'])
plt.plot(fake_biascorrected)

have to bias correct `sim` to be able to have quantiles to use in `adjust()`

In [ ]:
AIQPD.ds

In [ ]:
# QDM.ds

In [ ]:
AIQPD.adjust(fake_biascorrected['scen'], sim_q=fake_biascorrected.sim_q)

In [ ]:
# AIQPD.ds.af.isel(dayofyear=60).plot()

now compare to prototype notebook implementation of analog method 

compare for one day group without a window (so just 20 data points)

In [ ]:
ref_daygroups = ref.groupby('time.dayofyear')
hist_daygroups = hist.groupby('time.dayofyear')

In [ ]:
AIQPD_day = AnalogQuantilePreservingDownscaling(kind='+', nquantiles=20, group=sdba.Grouper("time.dayofyear"))

In [ ]:
AIQPD_day.train(ref, hist)
AIQPD_day

In [ ]:
day_inds = compute_day_ind(ref_daygroups[100])
ref_fine_100_test = hist_daygroups[100][day_inds.values]

In [ ]:
plt.plot(AIQPD_day.ds.af.sel(dayofyear=100).quantiles, hist_daygroups[100][day_inds.values] - ref_daygroups[100][day_inds.values])

AIQPD_day.ds.af.sel(dayofyear=100).plot()

made some actual adjustment factors. Make them at 1-degree because it's easier at first 

In [ ]:
store_filename = 'gs://impactlab-data/climate/downscaling/dc6-dev-75bpr/reference-rechunk.zarr' 
store = fs.get_mapper(store_filename, check=False)
ref_coarse = xr.open_zarr(store, consolidated=False)

In [ ]:
# ref_coarse

In [ ]:
ref_fine_1deg = ref_fine.rename({'latitude': 'lat', 'longitude': 'lon'}).reindex(lat=ref_coarse.lat, lon=ref_coarse.lon, method='nearest')

In [ ]:
ref_fine_1deg = ref_fine_1deg.chunk({'time': -1, 'lat': 10, 'lon': 10})

In [ ]:
ref_fine_1deg_slice = ref_fine_1deg.sel(time=slice('1995-01-01','2014-12-31'))
ref_coarse_slice = ref_coarse.sel(time=slice('1995-01-01','2014-12-31'))

In [ ]:
# west coast mask 
min_lon = -125
min_lat = 30
max_lon = -110
max_lat = 47

mask_lon = (ref_fine_1deg_slice.lon >= min_lon) & (ref_fine_1deg_slice.lon <= max_lon)
mask_lat = (ref_fine_1deg_slice.lat >= min_lat) & (ref_fine_1deg_slice.lat <= max_lat)

In [ ]:
ref_fine_1deg_slice_wc = ref_fine_1deg_slice.where(mask_lon & mask_lat, drop=True).load()

In [ ]:
ref_coarse_slice_wc = ref_coarse_slice.where(mask_lon & mask_lat, drop=True).load()

In [ ]:
AIQPD = AnalogQuantilePreservingDownscaling(kind='+', nquantiles=620, group=sdba.Grouper("time.dayofyear", window=31))

In [ ]:
AIQPD.train(ref_coarse_slice_wc['tasmax'], ref_fine_1deg_slice_wc['tmax'])
AIQPD

In [ ]:
# AIQPD.ds.af

In [ ]:
doy_af = AIQPD.ds.af.isel(dayofyear=220)

In [ ]:
doy_af.isel(quantiles=400).plot(robust=True)

treat first 620 data points as a day group to see if lack of data points is the problem

In [ ]:
AIQPD_day = AnalogQuantilePreservingDownscaling(kind='+', nquantiles=620, group=sdba.Grouper("time.dayofyear"))

need some smaller arrays for testing scaling up to 366 day groups. First use a single gridcell, then try a few gridcells. E.g. `tas_gc` and `tas_severalgc`

In [ ]:
x = np.arange(0, 620)
coarse_quantiles = x / float(len(x))

In [ ]:
u = tas_severalgc.groupby('time.dayofyear').map(compute_day_ind)

# tas_gc.quantile(q=coarse_quantiles)
tas_severalgc.rank(dim='time')
argsorted = tas_severalgc.argsort(axis=0)
tas_sorted = xr.apply_ufunc(np.take_along_axis, tas_severalgc, kwargs={"axis": 0, "indices": argsorted})

tas_severalgc.quantile(q=coarse_quantiles, dim='time') - tas_sorted[:620, :, :].values

test_sort = np.take_along_axis(tas_severalgc.values, tas_severalgc.argsort(axis=0).values, axis=0)